In [67]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
from PIL import Image

In [68]:
# Settings
device = 'cuda' if torch.cuda.is_available() else 'cpu'
size = 60
batch_size = 32
print(f'Device: {device}')

Device: cuda


In [47]:
# CustomDataset
class CustomDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_paths = []

        classes = os.listdir(self.root)
        for class_name in classes:
            class_path = os.path.join(self.root, class_name)
            if os.path.isdir(class_path):
                images = os.listdir(class_path)
                for image_name in images:
                    image_path = os.path.join(class_path, image_name)
                    self.image_paths.append((image_path, int(class_name.split('_')[0])))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path, label = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [48]:
# transform
transform_train = transforms.Compose([
    # transforms.RandomCrop(32, padding=4),
    transforms.Resize(size=[size, size]),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

transform_validation = transforms.Compose([
    transforms.Resize(size=[size, size]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

transform_test = transforms.Compose([
    transforms.Resize(size=[size, size]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

In [49]:
# Create custom datasets
dataset_folder = r'./dataset/'

train_folder = os.path.join(dataset_folder, 'CUSTOM_COFFEE_LEAF_DATASET_SPLITTED/train')
validation_folder = os.path.join(dataset_folder, 'CUSTOM_COFFEE_LEAF_DATASET_SPLITTED/validation')
test_folder = os.path.join(dataset_folder, 'CUSTOM_COFFEE_LEAF_DATASET_SPLITTED/test')

train_dataset = CustomDataset(root=train_folder, transform=transform_train)
validation_dataset = CustomDataset(root=validation_folder, transform=transform_validation)
test_dataset = CustomDataset(root=test_folder, transform=transform_test)

train_dataset_size = len(train_dataset)
validation_dataset_size = len(validation_dataset)
test_dataset_size = len(test_dataset)

print(f"Number of images in train dataset: {train_dataset_size}")
print(f"Number of images in validation dataset: {validation_dataset_size}")
print(f"Number of images in test dataset: {test_dataset_size}")

# Create data loaders
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Number of images in train dataset: 43813
Number of images in validation dataset: 12515
Number of images in test dataset: 6264


### Inspecting the checkpoint

In [50]:
checkpoint = torch.load('./checkpoint/coffee-leaf-diseases-custom-7-classes-densenet121-acc99.60-ckpt.t7', map_location='cpu')

keys = checkpoint.keys()
print(keys)

print(checkpoint['acc'])
print(checkpoint['epoch'])

keys = checkpoint['model'].keys()

for key in keys:
    print(key)

dict_keys(['model', 'optimizer', 'scaler', 'acc', 'epoch'])
99.60575386254662
10
module.features.conv0.weight
module.features.norm0.weight
module.features.norm0.bias
module.features.norm0.running_mean
module.features.norm0.running_var
module.features.norm0.num_batches_tracked
module.features.denseblock1.denselayer1.norm1.weight
module.features.denseblock1.denselayer1.norm1.bias
module.features.denseblock1.denselayer1.norm1.running_mean
module.features.denseblock1.denselayer1.norm1.running_var
module.features.denseblock1.denselayer1.norm1.num_batches_tracked
module.features.denseblock1.denselayer1.conv1.weight
module.features.denseblock1.denselayer1.norm2.weight
module.features.denseblock1.denselayer1.norm2.bias
module.features.denseblock1.denselayer1.norm2.running_mean
module.features.denseblock1.denselayer1.norm2.running_var
module.features.denseblock1.denselayer1.norm2.num_batches_tracked
module.features.denseblock1.denselayer1.conv2.weight
module.features.denseblock1.denselayer2.nor

### Loading the net

In [51]:
net = models.densenet121(weights='DEFAULT')
net.classifier = torch.nn.Linear(1024, 7)
net

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [52]:
# Load the checkpoint
#checkpoint = torch.load('./checkpoint/XXXXX.t7', map_location='cpu')

# Remove the "module" prefix from the keys
new_state_dict = {k.replace('module.', ''): v for k, v in checkpoint['model'].items()}

# Load the modified state_dict into the model
net.load_state_dict(new_state_dict)

<All keys matched successfully>

In [71]:
criterion = nn.CrossEntropyLoss()

def test(net, testloader):
    net = net.to(device)
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100.*correct/total

    return test_loss, acc

# def test(net, testloader):
#     net = net.to(device)
#     net.eval()
#     correct = 0
#     total = 0
#     predictions = []
#     ground_truth = []

#     with torch.no_grad():
#         for batch_idx, (inputs, targets) in enumerate(testloader):
#             inputs, targets = inputs.to(device), targets.to(device)
#             outputs = net(inputs)
#             _, predicted = outputs.max(1)
#             predictions.extend(predicted.cpu().numpy())
#             ground_truth.extend(targets.cpu().numpy())
#             total += targets.size(0)
#             correct += (predicted == targets).sum().item()  # Use element-wise comparison

#     accuracy = correct / total  # Calculate accuracy manually

#     return accuracy


In [72]:
test_loss, test_acc = test(net, testloader)
print(f'Test loss {(test_loss):.5f}, Test accuracy: {(test_acc):.5f}')

# accuracy = test(net, testloader)
# print(f'Test accuracy: {accuracy}')

Test loss 2.74459, Test accuracy: 99.79246
